<a href="https://colab.research.google.com/github/Luffy-Cloud-creator/Summarize_sales_data/blob/main/Summary_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder.appName('DF').getOrCreate()
spark

In [ ]:
df= spark.read.csv('/content/AmazonSalesData.csv',header= True, inferSchema= True)
df.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|    255.28|   159.42|    2533654.0| 1582243.5|    951410.5|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804|     205.7|   117.11|     576782.8| 328376.44|   248406.36|
|              Europe|              Russia|Of

In [64]:
null_counts = df.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+----------+--------+---------+----------+---------+----------+------------+---------+------+-------------+-----+-------------------+-----------------+----------------------------+----+--------------+
|Order date|Order id|Item Type|unit price|unit cost|units sold|total profit|Ship Date|Region|orders_in_72h|month|avg_units_per_month|margin_percentage|rank_by_profit_per_item_type|year|yearly_revenue|
+----------+--------+---------+----------+---------+----------+------------+---------+------+-------------+-----+-------------------+-----------------+----------------------------+----+--------------+
|         0|       0|        0|         0|        0|         0|           0|        0|     0|            0|    0|                  0|                0|                           0|   0|             0|
+----------+--------+---------+----------+---------+----------+------------+---------+------+-------------+-----+-------------------+-----------------+----------------------------+----+-----------

In [ ]:
region_and_country_with_profit = df.groupBy(col('Region')).sum('Total Profit')
region_with_highest_profit = region_and_country_with_profit.orderBy("sum(Total Profit)", ascending=False)
print(f"{region_with_highest_profit['Region']} Region has the highest profit with a total profit of {region_with_highest_profit['sum(Total Profit)']}")
country_with_highest_profit = df.filter(col('Region') == region_with_highest_profit['Region']) \
                                .groupBy(col('Country')).sum('Total Profit') \
                                .orderBy("sum(Total Profit)", ascending=False)
print(f"{country_with_highest_profit['Country']} country, under region {region_with_highest_profit['Region']} has the highest profit with a total profit of {country_with_highest_profit['sum(Total Profit)']}")


Column<'Region'> Region has the highest profit with a total profit of Column<'sum(Total Profit)'>
Column<'Country'> country, under region Column<'Region'> has the highest profit with a total profit of Column<'sum(Total Profit)'>


In [ ]:
orders = df.groupBy(window(col('Order Date'), '72 hours'))\
         .agg(count('Order ID').alias('count_order_id'))\
         .orderBy('count_order_id', ascending=False)


print(f"Highest number of orders ordered under a 72 hr period is {orders.first()}")


Highest number of orders ordered under a 72 hr period is None


In [83]:
from pyspark.sql.window import Window

# Average units sold per month
df = df.withColumn("month", month("order date"))
df = df.withColumn("avg_units_per_month",avg("units sold").over(Window.partitionBy("month")))

# Margin percentage (assuming you have 'unit_price' and 'unit_cost' columns)
df = df.withColumn("margin_percentage", ((col("unit price") - col("unit cost")) / col("unit price")) * 100)

# Calculate the overall lowest and highest margin percentage
lowest_overall_margin = df.agg(min("margin_percentage")).collect()[0][0]
highest_overall_margin = df.agg(max("margin_percentage")).collect()[0][0]

# Add these as new columns to every row of the DataFrame
df = df.withColumn("lowest_overall_margin", lit(lowest_overall_margin))
df = df.withColumn("highest_overall_margin", lit(highest_overall_margin))

# Top 3 highest value orders per item type
window_item_type = Window.partitionBy("item type").orderBy(col("total profit").desc())
df = df.withColumn("rank", dense_rank().over(window_item_type))
df = df.filter(col("rank") <= 3)

#Item most frequently sold on weekends
#df = df.withColumn("day_of_week", date_format("order date", "EEEE"))
df = df.withColumn("weekday", dayofweek(col("Order date")))
df = df.withColumn("is_weekend", col("weekday").isin([1, 7]))
df = df.withColumn("weekend_units_sold", when(col("is_weekend"), col("units sold")).otherwise(0))
Item_most_frequently_sold_on_weekends = df.filter(col("is_weekend")) \
                                           .groupBy(col("Item Type")) \
                                           .agg(sum('units sold').alias("total_weekend_units_sold")) \
                                           .orderBy("total_weekend_units_sold", ascending=False)

#Yearly Revenue
df = df.withColumn("year", year("order date"))
df = df.withColumn("yearly_revenue", sum("total profit").over(Window.partitionBy("year")))

df.show()

+-------------------+--------+---------+----------+---------+----------+------------+-------------------+------------------+-------------+-----+-------------------+------------------+----------------------------+----+--------------+----+---------------------+----------------------+-------+----------+------------------+
|         Order date|Order id|Item Type|unit price|unit cost|units sold|total profit|          Ship Date|            Region|orders_in_72h|month|avg_units_per_month| margin_percentage|rank_by_profit_per_item_type|year|yearly_revenue|rank|lowest_overall_margin|highest_overall_margin|weekday|is_weekend|weekend_units_sold|
+-------------------+--------+---------+----------+---------+----------+------------+-------------------+------------------+-------------+-----+-------------------+------------------+----------------------------+----+--------------+----+---------------------+----------------------+-------+----------+------------------+
|2023-03-18 09:00:00|  ORD008| Keyboa